In [ ]:
#Import all of the necessary packages
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd

We will now download the dataset consisting of the AAPL stock price over the last 10 years.

In [ ]:
#download dataframe here


We will now need to do some data preprocessing.

In [ ]:
#do preprocessing here

set index of dataset

In [ ]:
#setting date index of dataset

Calculate the simple moving average and ratio column

In [ ]:
#calculate simple moving average and the ratio here


Find percentiles and in regards to ratio and apply them to create a graph

now calculate the signals of the strategy based upon these percentiles

Calculate PNL

In [ ]:
def calc_pnl(df):
    position = 0 
    entry_price = 0 
    pnl = []  
    for index, row in df.iterrows():
        signal = row['Signal']
        price = row['Close']
        if position == 0: 
            if signal == 1:
                position = 1
                entry_price = price
                pnl.append(0)
            else:
                pnl.append(0)
        elif position == 1: 
            if signal == -1:
                if price < entry_price:
                    print(price, entry_price)
                pnl.append(price - entry_price)
                position = 0
            else:
                pnl.append(0)
    df['pnl'] = np.cumsum(pnl)
    return df
df = calc_pnl(df)

In [ ]:
#plot pnl
plt.plot(df['pnl'], label='PNL')
plt.title('Profit and Loss Over Time')
plt.show()

In [ ]:
#plot how our signals change over time
#For a better view on the decisions our algorithm makes
df['Buy'] = np.where(df.Signal == 1, df['Close'], np.nan)
df['Sell'] = np.where(df.Signal == -1, df['Close'], np.nan)
plt.title('Close Price w/ Buy and Sell Signals')
plt.plot(df['Close'], alpha=0.5, label = 'Close')
plt.plot(df['SMA'], alpha = 0.5, label='SMA')
plt.scatter(df.index, df.Buy, alpha = 1, color = 'green', label = 'Buy Signal', marker = '^')
plt.scatter(df.index, df.Sell, alpha = 1, color = 'red', label = 'Sell Signal', marker = '^')
plt.xlim(pd.to_datetime("1/1/2016"), pd.to_datetime("6/1/2019"))
plt.legend()
plt.show()

In [ ]:
#Check how different percentiles affect pnl
buy_percentile = 80 #Change with your own value
sell_percentile = 20 #Change with your own value
calc_signals(buy_percentile, sell_percentile)
calc_pnl(df)
plt.plot(df['pnl'], label='PNL')
plt.title('Profit and Loss Over Time')
plt.show()
print(df['pnl'].tail(1))

## Momentum Based Strategy

In [ ]:
df = pd.read_csv("data.csv")
df.rename(columns={" Close/Last":"Close", " Volume":"Volume", " Open":"Open", " High":"High", " Low":"Low"}, inplace=True)
bad_columns = ['Close', 'Open', 'High', 'Low']
for col in bad_columns:
    df[col] = df[col].apply(lambda x: float(x.strip(" ")[1:]))
df = df.set_index(pd.DatetimeIndex(df['Date'].values))
df = df.sort_index()
df.drop(labels=['Date', 'Open', 'High', 'Low', 'Volume'], axis=1, inplace=True)
print(df.head())

In [ ]:
## change the window size from 15 to other windows to check model acccuracy
price_diff = df['Close'].diff()
price_diff.dropna(inplace=True)

price_up = price_diff.copy()
price_down = price_diff.copy()

price_up[price_up < 0] = 0
price_down[price_down > 0] = 0
price_down *= (-1)
avg_up = price_up.rolling(15).mean()
avg_down = price_down.rolling(15).mean()
rsi = 100 * (avg_up /(avg_up + avg_down))
rsi.dropna(inplace=True)
q_25 = np.percentile(rsi, [25])
q_75 = np.percentile(rsi, [75])
print(q_25, q_75)

In [ ]:
plt.plot(rsi, label='RSI')
plt.axhline(q_25, c='red', label="25th percentile")
plt.axhline(q_75, c='green', label="75th percentile")
plt.legend()
plt.show()

In [ ]:
df['RSI'] = rsi
df['Signal'] = 0  # Default to hold
df['RSI'] = df['RSI'].fillna(50)
q_25 = df['RSI'].quantile(0.25)
q_75 = df['RSI'].quantile(0.75)
df.loc[df['RSI'] < q_25, 'Signal'] = 1  # Buy signal
df.loc[df['RSI'] > q_75, 'Signal'] = -1  # Sell signal
df = calc_pnl(df)
plt.plot(df['pnl'], label='PNL')
plt.title('Profit and Loss Over Time')
plt.show()

In [ ]:
#For a better view on the decisions our algorithm makes
df['Buy'] = np.where(df.Signal == 1, df['Close'], np.nan)
df['Sell'] = np.where(df.Signal == -1, df['Close'], np.nan)
plt.title('Close Price w/ Buy and Sell Signals')
plt.plot(df['Close'], alpha=0.5, label = 'Close')
plt.scatter(df.index, df.Buy, alpha = 1, color = 'green', label = 'Buy Signal', marker = '^')
plt.scatter(df.index, df.Sell, alpha = 1, color = 'red', label = 'Sell Signal', marker = '^')
plt.xlim(pd.to_datetime("1/1/2016"), pd.to_datetime("6/1/2019"))
plt.legend()
plt.show()

## We will now switch to the LSTM Strategy on the other python notebook